<a href="https://www.kaggle.com/code/pyliraghuram/xgb-regression-2?scriptVersionId=144445819" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Analyse  the data

In [ ]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull().sum()/len(test)*100

In [ ]:
test.isnull().sum()/len(test)*100

In [ ]:
train.duplicated().value_counts()

In [ ]:
test.duplicated().value_counts()

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
sns.heatmap(train.corr())

In [ ]:
sns.heatmap(test.corr())

In [ ]:
for i in train.columns:
    print(i)
    print('*--*'*10)
    print(train[i].value_counts(normalize = True,sort = True)*100)
    

In [ ]:
train.columns

# clean the data

In [ ]:
train =train.drop(['Id','Street', 'Utilities','LandSlope','Condition2','RoofMatl','CentralAir','LowQualFinSF','3SsnPorch','ScreenPorch','PoolArea','MiscVal'], axis=1)
test =test.drop(['Id','Street', 'Utilities','LandSlope','Condition2','RoofMatl','CentralAir','LowQualFinSF','3SsnPorch','ScreenPorch','PoolArea','MiscVal'], axis=1)

In [ ]:
for df in (train, test):
    df["n_bathrooms"] = df["BsmtFullBath"] + (df["BsmtHalfBath"]*0.5) + df["FullBath"] + (df["HalfBath"]*0.5)
    df["area_with_basement"]  = df["GrLivArea"] + df["TotalBsmtSF"]
    

In [ ]:
train = train.fillna(train.mean())
test = test.fillna(test.mean())

In [ ]:
train

In [ ]:
test

# making the data

In [ ]:
X_train = train.select_dtypes(include=['number']).copy()
X_train = X_train.drop(['SalePrice'], axis=1)
y_train = train["SalePrice"]
X_test  = test.select_dtypes(include=['number']).copy()

In [ ]:
X_train.head()

In [ ]:
for i in X_train.columns :
    print('-*-*'*10)
    print(i)
    print(X_train[i].isnull().sum())
    print('___+++__'*8)

In [ ]:
for i in X_test.columns :
    print('-*-*'*10)
    print(i)
    print(X_test[i].isnull().sum())
    print('___+++__'*8)

In [ ]:

print(y_train.isnull().sum())
print('___+++__'*8)

# feature scaling

In [ ]:
import xgboost as xg
from sklearn.preprocessing import RobustScaler


rrs =RobustScaler()
X_train= rrs.fit_transform(X_train)
X_test = rrs.transform(X_test)

In [ ]:
xgb = xg.XGBRegressor(eval_metric='rmsle')

# grid search CV

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(xgb,scoring = 'accuracy',cv = 5,param_grid  = {'max_depth' : [3,5],"n_estimators": [500, 600,650],"learning_rate": [0.01,.1, 0.015],
'gamma': [0.1, 0.2],'reg_lambda': [0.5, 1.0],})

# fit the model

gs.fit(X_train, y_train)

print("Best Parameters:", gs.best_params_)
print("Best Score:", gs.best_score_)

In [ ]:
print('Done')

In [ ]:
xgb = xg.XGBRegressor(gamma = 0.1, learning_rate  = 0.01, max_depth = 3, n_estimators= 500, reg_lambda = 0.1)
xgb.fit(X_train,y_train)
print(xgb.score(X_train,y_train)*100)

In [ ]:
solution = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
y_test = solution['SalePrice']

In [ ]:
prediction = xgb.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_log_error
RMSLE = np.sqrt( mean_squared_log_error(y_test, prediction) )
print("The score is %.5f" % RMSLE )

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
print("Mean Absolute Error:", mean_absolute_error(y_test, prediction))
print("Root Mean Squared Error:", np.sqrt(mean_absolute_error(y_test, prediction)))